<a href="https://colab.research.google.com/github/ViniciusCastillo/BootcampAlura_ProjetoModulo3/blob/main/Notebooks%5Cfuncoes%26bases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação de Funções, Importação e Tratamento Inicial dos Dados

Neste notebook temos as funções que serão utilizadas na análise, bem como imporatermos os dados, fazer um pré processamento para facilitar a análise no notebook principal do projeto

## Importanto bibliotecas que serão utilizadas

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

## Funções

### Função <font color='blue'>metricas</font><font size=3>(df_real, df_previsto, titulo)</font>
####Cálcula as métricas de comparação entre o dataframe real e o previsto

##### **Entradas e Saídas**

Informações de entrada:
* <font color='green'>df_real</font>: data frame real
* <font color='green'>df_previsto</font>: data frame previsto
* <font color='green'>titulo</font>: titulo da comparação que será realizada

Saídas:
* Printa o <font color='green'>título</font> da comparação, depois o resultado de 3 métricas:
> 1. **MAE**: Mean Absolute Error
> 1. **RMSE**: Root Mean Squared Error
> 1. **MAPE**: Mean Absolute Percentage Error

* Está função não retorna nenhum valor

In [2]:
def metricas (df_real, df_previsto, titulo):
  print ("\n"+ titulo)
  print ("Mean Absolute Error: %.2f" % mean_absolute_error(df_real['y'], df_previsto['yhat']))
  print ("Root Mean Squared Error: {:,.2f}".format(mean_squared_error(df_real['y'], df_previsto['yhat'])))
  print ("Mean Absolute Percentage Error: {:,.2f}".format(mean_absolute_percentage_error(df_real['y'], df_previsto['yhat'])))

### Função <font color='blue'>cria_DFs</font><font size=3>(dados, campo_y, inicio_teste, inicio_treino=<font color='yellow'>0</font>, campo_data=<font color='yellow'>'datahora'</font>)</font>
#### Divide uma base da dados entre uma base de treino e outra base de teste para criação de modelos preditivos, já no padrão do Facebook Prophet

##### **Entradas e Saídas**

Informações de entrada:
* <font color='green'>dados</font>: data frame que será dividido
* <font color='green'>campo_y</font>: campo do data frame que será utilizado como a varável ***'y'*** do modelo preditivo do Prophet
* <font color='green'>inicio_teste</font>: valor do registro inicial para a base de teste, consequentemente será utilizado como o fim da base de treino
* <font color='green'>inicio_treino</font>: valor do registro inicial para a base de treino, por padrão tem o valor <font color='yellow'>0</font>
* <font color='green'>campo_data</font>: campo do data frame que possuí a data e que será utilizado na varável ***'ds'*** do modelo preditivo do Prophet, por padrão tem o valor <font color='yellow'>'datahora'</font>

Saídas:
* Esta função retorna 2 informações:
> 1. <font color='purple'>df_treino</font>: data frame de treino
> 1. <font color='purple'>df_teste</font>: data fram de teste
* Estes data frames retornados estão com o seu index resetado
* Precisa ser atribuida para uma dupla de variáveis, por exemplo: *df_treino, df_teste = cria_DFs(dados, campo_y, inicio_teste)*

In [5]:
def cria_DFs (dados, campo_y, inicio_teste, inicio_treino=0, campo_data='datahora'):
  df_teste = pd.DataFrame()
  # a base de teste será sempre do início enviado até o final da base de dados
  df_teste['ds'] = dados[campo_data][inicio_teste:]
  df_teste['y'] = dados[campo_y][inicio_teste:]
  df_teste = df_teste.reset_index(drop=True)
  df_treino = pd.DataFrame()
  # a base de treino terá como fim o início do teste enviado
  df_treino['ds'] = dados[campo_data][inicio_treino:inicio_teste]
  df_treino['y'] = dados[campo_y][inicio_treino:inicio_teste]
  df_treino = df_treino.reset_index(drop=True)
  return df_treino, df_teste

### Função <font color='blue'>modelar</font><font size=3>(df_treino, df_teste, y_label, show_change_points=<font color='yellow'>False</font>, **todos os parâmetros da função Prophet da biblioteca Facebook Prophet)</font>
#### Cria o modelo preditivo da biblioteca Facebook Prophet, retorna o modelo e a base de previsão. Fora isso, exibe o gráfico do modelo, incluindo a comparação com a base de teste

##### **Entradas e Saídas**

Informações de entrada:
* <font color='green'>df_treino</font>: data frame que será utilizado para trienar o modelo. **Obersevação**: lembrar que o se o parâmetro *growth* for alterado para *'logistic'* é necessário incluir o campo *'cap'<font size=1><sup>(1)</sup></font>* neste data frame antes de chamar esta função
* <font color='green'>df_teste</font>: data frame que será plotado como comparação para a projeção
* <font color='green'>y_label</font>: valor que será plotado como título do eixo Y do gráfico
* <font color='green'>show_change_points</font>: se passar o valor <font color='grey'>**True**</font> plota os change points no gráfico, por padrão o valor é <font color='yellow'>False</font>
* <font color='green'>** parâmetros do Prophet</font>: pode-se passar qualquer parâmentro da função Prophet da biblioteca Facebook Prophet

<font size=2> <font size=1><sup>(1)</sup></font> *'cap' é a definição do valor máximo esperado na função logística e precisa ter valor em todas as linhas do data frame. [Link para página do Prophet sobre isso](https://facebook.github.io/prophet/docs/saturating_forecasts.html#forecasting-growth)*</font>

Saídas:
* Esta função retorna 2 informações:
> 1. <font color='purple'>modelo</font>: o modelo calculado pelo Prophet
> 1. <font color='purple'>previsao</font>: a base de previsão calculada pelo Prophet. Vale destacar que os dados futuros terão o mesmo tamanho que o <font color='green'>df_teste</font> que foi passado
* Ela também plota o gráfico do modelo previsto (<font color='purple'>previsao</font> como uma linha azul), bem como as informações reais, tanto do <font color='green'>df_treino</font> (linha cinza) quanto <font color='green'>df_teste</font> (linha laranja), para comparação visual
* Caso tenha optado pela opção de mostrar os change points, eles irão aparecer no gráfico também

In [8]:
def modelar (df_treino, df_teste, y_label, show_change_points=False, growth='linear', changepoints=None, n_changepoints=25, changepoint_range=0.8, 
             yearly_seasonality='auto', weekly_seasonality='auto', daily_seasonality='auto', holidays=None, seasonality_mode='additive', 
             seasonality_prior_scale=10.0, holidays_prior_scale=10.0, changepoint_prior_scale=0.05, mcmc_samples=0, interval_width=0.8, 
             uncertainty_samples=1000, stan_backend=None):
  # cria o modelo com os parâmetros escolhidos
  modelo = Prophet(growth, changepoints, n_changepoints, changepoint_range, yearly_seasonality, weekly_seasonality, daily_seasonality, 
                   holidays, seasonality_mode, seasonality_prior_scale, holidays_prior_scale, changepoint_prior_scale, mcmc_samples, 
                   interval_width, uncertainty_samples, stan_backend)
  # treina o modelo com o df_treino
  modelo.fit(df_treino)
  # cria o df para previsão futura do mesmo tamanho que o df_teste
  df_futuro = modelo.make_future_dataframe(periods=len(df_teste))
  # se estiver projetando um modelo logístico, inclui o 'cap' no df_futuro
  if growth=='logistic': df_futuro['cap'] = df_treino['cap'].iloc[0];
  # preve os pontos futuros com base no modelo treinado anteriormente
  previsao = modelo.predict(df_futuro)
  # plota a previsão, onde o nome do eixo y é igual ao que foi passado na função
  fig = modelo.plot(previsao, xlabel='Data', ylabel=y_label, figsize=(15,8))
  # se estiver definido show_change_points como True, plota os change points
  if show_change_points: a = add_changepoints_to_plot(fig.gca(), modelo, previsao);
  # plota o df_teste como uma linha pontilhada laranja
  plt.plot(df_teste.ds, df_teste.y, ':', marker='.', color='orange')
  # plota o df_treino como uma linha pontilhada cinza
  plt.plot(df_treino.ds, df_treino.y, ':', color="grey")
  # plt.xticks(color='black')
  # plt.yticks(color='black')
  return modelo, previsao # retorna as informações do modelo que foi treinado e o df da previsão criada

## Bases

### lista de feriados

#### lista de feriados da cidade de São Paulo de 2020

In [10]:
feriados_sp_2020 = [
'2020-02-25',
'2020-02-26',
'2020-04-10',
'2020-02-25',
'2020-02-26',
'2020-04-10',
'2020-04-21',
'2020-05-01',
'2020-05-20',
'2020-05-21', 
'2020-05-22',
'2020-05-25',
'2020-06-11',
'2020-09-07',
'2020-10-12',
'2020-11-02',
'2020-11-15',
'2020-12-25'
]

In [11]:
feriados_sp_2020 = pd.DataFrame({
  'holiday': 'sp_2020',
  'ds': pd.to_datetime(feriados_sp_2020)
})

#### lista de feriados da cidade de São Paulo de 2021

In [22]:
feriados_sp_2021 = [
'2021-01-01',
'2021-01-25',
'2021-02-15',
'2021-02-16',
'2021-02-17',
'2021-03-26',
'2021-03-29',
'2021-03-30',
'2021-03-31',
'2021-04-01',
'2021-04-02',
'2021-04-21',
'2021-05-01',
'2021-06-03',
'2021-07-09',
'2021-09-07',
'2021-10-12',
'2021-11-02',
'2021-11-15',
'2021-12-25'
]

In [23]:
feriados_sp_2021 = pd.DataFrame({
  'holiday': 'sp_2021',
  'ds': pd.to_datetime(feriados_sp_2021)
})


#### lista consolida de feriados da cidade de São Paulo 2020 e 2021

In [24]:
feriados_sp = pd.concat((feriados_sp_2020, feriados_sp_2021))
print(feriados_sp.head())
print(feriados_sp.tail())

   holiday         ds
0  sp_2020 2020-02-25
1  sp_2020 2020-02-26
2  sp_2020 2020-04-10
3  sp_2020 2020-02-25
4  sp_2020 2020-02-26
    holiday         ds
15  sp_2021 2021-09-07
16  sp_2021 2021-10-12
17  sp_2021 2021-11-02
18  sp_2021 2021-11-15
19  sp_2021 2021-12-25


### dados de covid Brasil.io